<a href="https://colab.research.google.com/github/TMMeme/Research/blob/main/QConv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 3.7 MB/s 
     |████████████████████████████████| 13.6 MB 12.2 MB/s 
     |████████████████████████████████| 1.6 MB 46.5 MB/s 
     |████████████████████████████████| 108 kB 59.3 MB/s 


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import numpy as np
import torch
from torch import Tensor 

In [ ]:
#変数の定義
seqlength=100
kernel=13 #フィルターの長さ＝motifの長さ=量子ビットの数に等しい
stride = 1

In [ ]:
#入力をTensor型で準備する

input_tensor_A=torch.randint(low=0, high=2, size=(seqlength, ))
input_tensor_T=torch.randint(low=0, high=2, size=(seqlength, ))
input_tensor_G=torch.randint(low=0, high=2, size=(seqlength, ))
input_tensor_C=torch.randint(low=0, high=2, size=(seqlength, ))

input_tensor_list = [input_tensor_A, input_tensor_T, input_tensor_G, input_tensor_C]

In [ ]:
#deviceの定義
dev = qml.device('default.qubit',wires=kernel)#今とりあえずlightnin.gpu使ってるけど必要に応じてdefault.qubitに変えてもいい
seed = np.random.randint(low=0, high=10e6) #ランダム量子回路形を指定する乱数シード

@qml.qnode(dev)
def circuit(feature_vector, weights):
  #Encoding of classical input values
  qml.AngleEmbedding(features=feature_vector, wires=range(kernel), rotation='Y')
  #RandomQuantumCircuit
  qml.RandomLayers(weights, wires=range(kernel))
  return [qml.expval(qml.PauliZ(i)) for i in range(kernel)]

In [ ]:
#QConv関数を定義する
def QConv1d(input_tensor_list, seqlength, kernel, weights):
  feature_map_A = []
  feature_map_T = []
  feature_map_G = []
  feature_map_C = []
  feature_map_list=[feature_map_A,feature_map_T,feature_map_G,feature_map_C]

  for i in range(4): 
    for j in range(0, seqlength-kernel+1, stride):
      feature_map_list[i].append(np.sum(circuit(feature_vector=input_tensor_list[i][j:j+kernel],weights=weights)).item())
  
  output_tensor_A = torch.Tensor(feature_map_A)
  output_tensor_T = torch.Tensor(feature_map_T)
  output_tensor_G = torch.Tensor(feature_map_G)
  output_tensor_C = torch.Tensor(feature_map_C)
  output_tensor=(output_tensor_A + output_tensor_T + output_tensor_G + output_tensor_C).reshape(1, 88)
  return output_tensor

QConv1dの定義できた。
次やりたいこと

1.   64枚のマップをcatする
2.   GPUによる高速化（lightning.gpuによる高速化）

In [ ]:
#64枚分のフィルタにQConvを適用し、最終出力をtorch.Size([64, 88])の2階テンソルで得る
output_tensor_list = []
num = 0
while num < 64:
  samp_weights = torch.randn(1,20)
  weights=samp_weights
  output = QConv1d(input_tensor_list=input_tensor_list, seqlength=seqlength, kernel=kernel, weights=weights)
  output_tensor_list.append(output)
  num +=1

final_output = torch.cat(output_tensor_list, dim=0)
print(final_output.size())

torch.Size([64, 88])


In [ ]:
print(final_output)

tensor([[16.2862, 16.8249, 17.0872,  ..., 18.1956, 17.8237, 16.0999],
        [23.2651, 25.0968, 24.5123,  ..., 22.7656, 23.2568, 24.0142],
        [26.2963, 27.6409, 29.5095,  ..., 26.8362, 27.3919, 26.0408],
        ...,
        [30.8843, 31.5900, 31.5891,  ..., 30.1260, 30.3540, 31.1829],
        [24.0873, 29.1625, 26.8224,  ..., 26.3668, 25.0328, 26.8384],
        [25.6456, 30.1759, 28.7716,  ..., 26.9595, 25.4548, 28.0954]])
